<a href="https://colab.research.google.com/github/utkarshg1/DL-2pm-4pm-Weekend/blob/main/Pydantic_SIP_estimator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pydantic import BaseModel, Field
from datetime import date

In [2]:
class Retirement(BaseModel):

  monthly_expense: float = Field(gt=0, description="Monthly expenses")
  dob: date = Field(le=date.today(), description="Date of birth")
  retire_age: int = Field(default=60, description="Retirement Age")
  inflation_rate: float = Field(ge=0.0, le=100.0, default=8.0 , description="Inflation rate in percent")
  withdrawal_rate: float = Field(ge=0.0, le=100.0, default=3.0 , description="Withdrawal rate in percent")
  expected_return: float = Field(ge=0.0, le=100.0, default=12.0 , description="Expected return in percent")

  def get_age(self):
    today = date.today()
    age = today.year - self.dob.year - ((today.month, today.day) < (self.dob.month, self.dob.day))
    return age

  def get_years_to_retire(self):
    age = self.get_age()
    years_to_retire = self.retire_age - age
    if years_to_retire <= 0:
      raise ValueError("Years to retire cannot be negative")
    return years_to_retire

  def get_total_corpus(self):
    annual_expense = self.monthly_expense*12
    years_to_retire = self.get_years_to_retire()
    future_value = annual_expense*(1 + self.inflation_rate/100)**years_to_retire
    return future_value/(self.withdrawal_rate/100)

  def estimate_sip(self):
    i = self.expected_return/1200
    n = self.get_years_to_retire()*12
    m = self.get_total_corpus()
    x = (1 + i)**n - 1
    y = (x/i)*(1+i)
    return m/y

In [3]:
p1 = Retirement(monthly_expense=1_00_000, dob=date(1994, 12, 16))

In [4]:
p1

Retirement(monthly_expense=100000.0, dob=datetime.date(1994, 12, 16), retire_age=60, inflation_rate=8.0, withdrawal_rate=3.0, expected_return=12.0)

In [5]:
p1.get_age()

29

In [6]:
p1.get_years_to_retire()

31

In [7]:
p1.get_total_corpus()

434706777.6079731

In [9]:
p1.estimate_sip()

108938.02247692372